In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10.0, 10.0)


In [2]:
import pandas as pd
import pysal as ps
import numpy as np
'''
Arguments
---------
dbfile  : DBF file - Input to be imported
upper   : Condition - If true, make column heads upper case
'''
def dbf2DF(dbfile, upper=True): #Reads in DBF files and returns Pandas DF
    db = ps.open(dbfile) #Pysal to open DBF
    d = {col: db.by_col(col) for col in db.header} #Convert dbf to dictionary
    pandasDF = pd.DataFrame(db[:]) #Convert to Pandas DF
    pandasDF.columns=db.header
    pandasDF.dropna(inplace=True)
    return pandasDF

def getPercent(dataset,string):
    '''
    Arguments
    ---------
    dataset  : pandas datafra- Input to produce the % column on
    string   : String or list of strings - variable or variables to convert to %
    '''
    mask = (dataset.iloc[:,dataset.shape[1]-1] == 0)
    dataset = dataset.loc[~mask,:]
    variable = np.zeros(dataset.shape[0])
    
    #if me need to aggregate more columns in the same category:
    if len(string) > 1:
        variable = dataset.loc[:,string].sum(axis=1)
        
    else:
        variable = dataset[string].iloc[:,0]
    
    newVariableName = string[0].split('_')[0]
    
    dataset[newVariableName + '_p'] = variable/dataset.iloc[:,dataset.shape[1]-1] * 100
    dataset[newVariableName + '_p'] = dataset[newVariableName + '_p'].map(lambda x: round(x,1))
    dataset = dataset.loc[:,['REDCODE',newVariableName + '_p']]
    dataset.dropna(inplace=True)
    try:
        dataset[newVariableName + '_q'] = pd.qcut(dataset[newVariableName + '_p'], 10,labels=False)
        dataset[newVariableName + '_q'] = dataset[newVariableName + '_q'].map(lambda x: int(x))
        return dataset
    except ValueError:
        print 'Can not be cut in 10 quantiles'

# Cargo data

In [3]:
#chequeo cuantos radios coinciden 
radiosRedatam = dbf2DF('data/redatam/MNI.DBF')
print 'hay %d radios en el DBF de REDATAM' % (radiosRedatam.shape[0])
radiosRedatam.head()

hay 52382 radios en el DBF de REDATAM


,REDCODE,MNI_1,MNI_2,MNI_3,MNI_4,MNI_5,MNI_6,MNI_7,MNI_8,MNI_9,MNI_T
0,020010101,11,31,39,73,51,15,8,12,16,256
1,020010201,23,81,106,64,118,4,7,6,11,420
2,020010202,24,72,103,46,89,2,7,2,6,351
3,020010203,21,52,67,44,88,5,6,3,14,300
4,020010204,28,78,148,74,179,7,6,1,22,543


In [4]:
#8 es universitario completo
data = getPercent(dataset = radiosRedatam,string = ['MNI_8'])
data.head()

/home/pipe/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/pipe/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,REDCODE,MNI_p,MNI_q
0,020010101,4.7,5
1,020010201,1.4,3
2,020010202,0.6,1
3,020010203,1.0,2
4,020010204,0.2,0


# Cargo carto

In [5]:
#radios de caba y pba sacados de cartografia oficial de indec
mapa = gpd.GeoDataFrame.from_file('data/shapes/agba/agba.shp')
print mapa.shape
mapa.head()

(13446, 13)


,AREA,DEPARTAMEN,DEPTO,FRAC,LINK,PERIMETER,PROV,RADIO,RADPAIS_,RADPAIS_ID,REDCODE,TIPO,geometry
0,16948780.0,06252,252,01,062520101,26902.490,06,01,26690,26690,062520101,M,"POLYGON ((4158793.5 6185928, 4158824.25 618588..."
1,12351870.0,06252,252,01,062520102,30469.020,06,02,26706,26707,062520102,M,"POLYGON ((4159777 6185123.5, 4159847 6185160, ..."
2,11063170.0,06252,252,02,062520208,25350.040,06,08,26721,26723,062520208,U,"POLYGON ((4161392.5 6184536.5, 4161399.75 6184..."
3,796720.3,06252,252,02,062520201,6843.383,06,01,26743,26743,062520201,U,"POLYGON ((4164165.5 6183091, 4164223.5 6183021..."
4,2996825.0,06252,252,02,062520202,8393.760,06,02,26749,26749,062520202,U,"POLYGON ((4166774.25 6182343.5, 4166960.5 6182..."


 # Mergeo

In [6]:
mapa = mapa.merge(data,on='REDCODE')

In [8]:
mapa.to_file('data/shapes/agbaEduc')